# 🚚 Fase 05: Logistics & Delivery Performance
---
**Objetivo**: Analizar la eficiencia logística y el cumplimiento de promesas de entrega.

## 1. Carga de Datos
* Objetivo: Preparar el entorno y cargar los datasets necesarios para el análisis logístico.
* Datasets: orders_cleaned.csv y order_items_totals.csv.

In [3]:
import pandas as pd
import duckdb
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
path_processed = "../data/processed"
# Convertir strings a objetos `datetime`:

# Carga de datos
orders = pd.read_csv(f'{path_processed}/orders_cleaned.csv',
                        parse_dates=['order_purchase_timestamp', 
                                      'order_approved_at', 
                                      'order_delivered_carrier_date', 
                                      'order_delivered_customer_date', 
                                      'order_estimated_delivery_date'])
order_items = pd.read_csv(f"{path_processed}/order_items_totals.csv")

print(f"Dataset de órdenes cargado: {orders.shape[0]} registros.")

Dataset de órdenes cargado: 99441 registros.


In [5]:
orders.info()
orders.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
 8   delivery_time_days             96476 non-null  float64       
 9   delta_estimated_days           96476 non-null  float64       
dtypes: datetime64[ns](5), float64(2), object(3)
memory usage: 7.6+ MB


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_time_days,delta_estimated_days
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8.0,-8.0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,13.0,-6.0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9.0,-18.0


In [6]:
order_items.info()
order_items.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98666 entries, 0 to 98665
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   order_id             98666 non-null  object 
 1   total_items_price    98666 non-null  float64
 2   total_items_freight  98666 non-null  float64
 3   order_grand_total    98666 non-null  float64
dtypes: float64(3), object(1)
memory usage: 3.0+ MB


,order_id,total_items_price,total_items_freight,order_grand_total
0,00010242fe8c5a6d1ba2dd792cb16214,58.9,13.29,72.19
1,00018f77f2f0320c557190d7a144bdd3,239.9,19.93,259.83
2,000229ec398224ef6ca0657da4fc703e,199.0,17.87,216.87


## 2. Segmentación del Desempeño Logístico
* **Objetivo:** Clasificar los pedidos según el cumplimiento de la promesa de entrega.
* **Lógica:** * Pedidos con `delta_estimated_days` > 0: **Retrasados**.
    * Pedidos con `delta_estimated_days` <= 0: **A tiempo / Anticipados**.

In [20]:
# 1. Aseguro la creación de la categoría basada en el delta que ya tengo
orders['delivery_status'] = orders['delta_estimated_days'].apply(
    lambda x: 'Retrasado' if x > 0 else 'A tiempo / Anticipado'
)

# 2. Ahora sí, cuento cuántos hay de cada categoría (Status)
resumen_logistica = orders['delivery_status'].value_counts(normalize=True) * 100

print("--- Análisis de Cumplimiento de Entrega ---")
print(resumen_logistica)

# 3. Confirmo el número absoluto de retrasos
total_retrasos = (orders['delivery_status'] == 'Retrasado').sum()
print(f"\nTotal de pedidos con retraso: {total_retrasos}")

--- Análisis de Cumplimiento de Entrega ---
delivery_status
A tiempo / Anticipado    93.428264
Retrasado                 6.571736
Name: proportion, dtype: float64

Total de pedidos con retraso: 6535


#### 📝 Conclusión del análisis de la segmentación del desempeño logístico:
Insight de Entrega: El sistema logístico de Olist cumple con la promesa de entrega en el 93.4 % de los casos. Sin embargo, existe un 6.5 % de pedidos (6,535 órdenes) que llegan después de la fecha estimada, lo cual es un volumen crítico que impacta directamente en la satisfacción del cliente y posibles reclamos.